# Курсовой проект по курсу "Рекомендательные системы."

В данном курсовом проекте перед нами стоит задача выдачи  
рекомендаций товаров пользователям из тестового датасета, исходя  
из имеющихся данных о фактических покупках пользователей в  
обучающем датасете. Также даны некоторые признаки пользователей  
и товаров.  
Целью является получение метрики precision@5 более 0.235 на тестовом  
датасете.

## Загрузка библиотек и скриптов

In [1]:
from catboost import CatBoostRanker
import numpy as np
import pandas as pd
import sys
sys.path.append('./Modules/')

# свои модули
from metrics import precision_at_k, recall_at_k
from recommenders import MainRecommender
from utils import prefilter_items

## Загрузка датасетов

In [2]:
data_train = pd.read_csv('Data/retail_train.csv')
data_test = pd.read_csv('Data/retail_test.csv')
item_features = pd.read_csv('Data/product.csv')
user_features = pd.read_csv('Data/hh_demographic.csv')

## Задание констант

In [3]:
ITEM_COL = 'item_id'
USER_COL = 'user_id'
ACTUAL_COL = 'actual'
N_CANDIDATES = 50
N_RANGED = 5

Приведём датасеты к общему виду:

In [4]:
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
user_features.rename(columns={'household_key': USER_COL }, inplace=True)

## Разделение на обучение и валидацию

Разделим следующим образом:  
- модель первого этапа (предожение кандидатов) обучим на всём датасете,  
кроме 6 последних недель, валидировать её будем на оставшейся части;  
- модель второго этапа (ранжирование кандидатов) будем обучать на 6  
последних неделях.

In [5]:
# -- давние -- | -- 6 недель 

VAL_MATCHER_WEEKS = 6

In [6]:
# данные для тренировки matching модели
data_train_matcher = data_train[data_train['week_no'] <= (data_train['week_no'].max() - VAL_MATCHER_WEEKS)]

# данные для валидации matching модели
data_val_matcher = data_train[data_train['week_no'] > (data_train['week_no'].max() - VAL_MATCHER_WEEKS)]

# данные для тренировки ranking модели
data_train_ranker = data_val_matcher.copy()

In [7]:
data_train_matcher.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


## Префильтрация товаров

Отберём товары согласно нашей функции из модуля utils.py.  
А именно:  
- убираем товары, которые не продавались за последние 12 месяцев;  
- убираем некоторые категории товаров;  
- убираем слишком дешевые и слишком дорогие товары;  
- оставляем только топ-5000 популярных товаров.

Следует отметить, что исключение самых дешёвых товаров почти  
наверняка понизит нашу целевую метрику. В то же время, было  
принято решение отказаться от исключения топ-100 самых популярных  
товаров. Будем стараться искать баланс между метрикой и здравым  
смыслом, так как, с одной стороны, нам нужно получить метрику  
не хуже определённого значения, а с другой стороны, рекомендация  
товаров, которые пользователь и так купит, не несёт практического  
смысла.

In [8]:
n_items_before = data_train_matcher['item_id'].nunique()

data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, n_popular=5000)

n_items_after = data_train_matcher['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 85828 to 5001


Уберем из валидации тех пользователей, которых нет в обучении,  
так как "холодных" пользователей мы здесь не обрабатываем,  
согласно допущению в условии курсового проекта:

In [9]:
common_users = data_train_matcher.user_id.unique()

data_val_matcher = data_val_matcher.loc[data_val_matcher.user_id.isin(common_users)]
data_train_ranker = data_train_ranker.loc[data_train_ranker.user_id.isin(common_users)]

Напишем функции для оценки метрик recall@k и precision@k:

In [10]:
def calc_recall(df_result, top_k):
    for col_name in df_result.columns[2:]:
        yield col_name, df_result.apply(lambda row: recall_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

In [11]:
def calc_precision(df_result, top_k):
    for col_name in df_result.columns[2:]:
        yield col_name, df_result.apply(lambda row: precision_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

## Базовое решение (MVP)

Инициализируем наш recommender:

In [12]:
recommender = MainRecommender(data_train_matcher)

Подготовим тестовый датасет:

In [13]:
result_test = data_test.groupby(USER_COL, sort=False)[ITEM_COL].unique().reset_index()
result_test.columns=[USER_COL, ACTUAL_COL]
result_test.head(2)

,user_id,actual
0,1340,"[912987, 819255, 834117, 866227, 889362, 89608..."
1,588,"[1024426, 6534178, 9673270, 826842, 833025, 85..."


Посмотрим, есть ли в тестовом датасете пользователи, которых  
нет на трейне:

In [14]:
result_test.loc[~result_test[USER_COL].isin(data_train_matcher[USER_COL].unique())]

,user_id,actual
1220,2325,"[849274, 863885, 872137, 877913, 883932, 96520..."


Нашли одного такого пользователя, удалим его:

In [15]:
result_test = result_test.loc[result_test[USER_COL].isin(data_train_matcher[USER_COL].unique())]

За базовое решение примем генерацию кандидатов из самых  
популярных во всём тестовом датасете. У каждого пользователя  
будут одинаковые рекомендации:

In [16]:
result_test['top_popular'] = result_test[USER_COL].apply(lambda x: recommender.overall_top_purchases[:N_CANDIDATES])

In [17]:
result_test.head(2)

,user_id,actual,top_popular
0,1340,"[912987, 819255, 834117, 866227, 889362, 89608...","[1029743, 995242, 1106523, 981760, 1133018, 95..."
1,588,"[1024426, 6534178, 9673270, 826842, 833025, 85...","[1029743, 995242, 1106523, 981760, 1133018, 95..."


Оценим метрику таких рекомендаций:

In [18]:
baseline_metric = tuple(*calc_precision(result_test, N_RANGED))[1]
baseline_metric

0.12611464968152866

## Этап 1:
### Подбор кандидатов и сравнение их качества,  
### выбор модели подбора кандидатов

Теперь инициализируем наш recommender с параметрами для BM25 взвешивания,  
полученные эмпирическим путём:

In [19]:
recommender = MainRecommender(data_train_matcher, K1=1, B=0.3)

Подготовим датасет для оценки качества кандидатов:

In [20]:
result_eval_matcher = data_val_matcher.groupby(USER_COL, sort=False)[ITEM_COL].unique().reset_index()
result_eval_matcher.columns=[USER_COL, ACTUAL_COL]
result_eval_matcher.head(2)

,user_id,actual
0,843,"[845193, 865891, 883404, 904375, 923746, 93663..."
1,2223,"[7155012, 14077656, 936753, 941856, 1103105, 1..."


Создадим кандидатов методами собственных рекомендаций,  
als, похожих пользователей и похожих товаров:

In [21]:
def make_recommendations(df_result, rec_name_model, N=50):
    rec_name = rec_name_model[0]
    rec_model = rec_name_model[1]
    df_result[rec_name] = df_result[USER_COL].apply(lambda x: rec_model(x, N=N))

In [22]:
own_rec = ('own_recs', recommender.get_own_recommendations)
als_rec = ('als_recs', recommender.get_als_recommendations)
sim_user_rec = ('similar_user_recs', recommender.get_similar_users_recommendation)
sim_item_rec = ('similar_item_recs', recommender.get_similar_items_recommendation)

In [23]:
%%time

for rec in (own_rec, als_rec, sim_user_rec, sim_item_rec):
    make_recommendations(result_eval_matcher, rec, N=N_CANDIDATES)

CPU times: user 6min 41s, sys: 6min 58s, total: 13min 40s
Wall time: 1min 18s


Также сделаем смесь собственных рекомендаций и top-popular:

In [24]:
make_recommendations(result_eval_matcher, ('own+top_pop', recommender.get_own_recommendations), N=N_CANDIDATES//2)

In [25]:
def fill_with_tops(column, N=5):
    
    tops = np.array(recommender.overall_top_purchases)
    recs = np.array(column)
    mask = np.isin(tops, recs, invert=True)
    tops = tops[mask]
    
    return np.append(recs, tops[:N])

In [26]:
result_eval_matcher['own+top_pop'] = result_eval_matcher['own+top_pop']. \
        apply(lambda row: fill_with_tops(row, N=N_CANDIDATES//2))

In [27]:
len(result_eval_matcher.iloc[0]['own+top_pop'])

50

Посмотрим на recall@k кандидатов, полученных разными способами:

In [28]:
sorted(calc_recall(result_eval_matcher, N_CANDIDATES), key=lambda x: x[1], reverse=True)

[('own_recs', 0.11465071065557855),
 ('own+top_pop', 0.10104838811379524),
 ('als_recs', 0.08215630905471297),
 ('similar_user_recs', 0.05556668595366006),
 ('similar_item_recs', 0.04906370635485423)]

Целевая метрика моделей первого этапа:

In [29]:
sorted(calc_precision(result_eval_matcher, N_RANGED), key=lambda x: x[1], reverse=True)

[('own_recs', 0.3377612633534603),
 ('own+top_pop', 0.3377612633534603),
 ('als_recs', 0.18532280538783094),
 ('similar_item_recs', 0.10487691593125871),
 ('similar_user_recs', 0.09837436135624711)]

<ins>Вывод:</ins>  
По качеству кандидатов (recall@k) лучшими оказались способы  
генерации на основе собственных рекомендаций (модель ItemItemRecommender  
с параметром K=1): среди предложенных кандидатов у них оказалось больше  
всего релевантных позиций. Третье место заняла модель ALS.  
По целевой метрике (precision@5) ситуация та же. Причём метрика моделей  
собственных рекомендаций оказалась настолько большой, что вполне вероятно,  
что на тестовых данных эти модели превысят необходимую метрику в 0.235  
даже без применения ранжирования. Такие высокие показатели этих моделей  
были достигнуты за счёт подбора параметров к модели взвешивания BM25.  
Вроде, на этом можно было бы и заканчивать.  
Однако мы не будем применять модели собственных рекомендаций ввиду того,  
что практический смысл таких рекомендаций невелик: это зачастую товары,  
которые пользователь и так покупает без всяких рекомендаций. Также интересно  
было бы дальше поработать с другой моделью и попытаться повысить метрику за счёт  
применения ранжирования и создания новых признаков.  
Учитывая вышесказанное, далее будем работать с моделью ALS.

## Этап 2
### Обучение модели ранжирования

Подготовим данные для обучения:

In [30]:
df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
df_match_candidates.columns = [USER_COL]

Произведём отбор кандидатов методом ALS:

In [31]:
make_recommendations(df_match_candidates, ('candidates', recommender.get_als_recommendations), N=N_CANDIDATES)

In [32]:
df_match_candidates.head(3)

,user_id,candidates
0,843,"[8090521, 1106523, 5568378, 1092026, 8090537, ..."
1,2223,"[986912, 844165, 1085604, 1005186, 879755, 109..."
2,278,"[885863, 9527160, 1137775, 5568378, 1101173, 1..."


Разворачиваем кандидатов, чтобы в каждой строке был только один:

In [33]:
df_items = df_match_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
df_items.name = ITEM_COL

In [34]:
df_match_candidates = df_match_candidates.drop('candidates', axis=1).join(df_items)

In [35]:
df_match_candidates.head()

,user_id,item_id
0,843,8090521
0,843,1106523
0,843,5568378
0,843,1092026
0,843,8090537


Подготовим обучающий сет для ранжирования:

In [36]:
df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL]].copy()
df_ranker_train['target'] = 1  # тут только фактические покупки 

In [37]:
df_ranker_train

,user_id,item_id,target
2211065,843,845193,1
2211066,843,865891,1
2211067,843,883404,1
2211068,843,904375,1
2211069,843,923746,1
...,...,...,...
2396799,1613,16102849,1
2396800,1001,13217063,1
2396801,1001,13217800,1
2396802,1167,6410462,1


Добавляем кандитатов в датасет в качестве нулей:

In [38]:
df_ranker_train = df_match_candidates.merge(df_ranker_train, on=[USER_COL, ITEM_COL], how='left')

# чистим дубликаты
df_ranker_train = df_ranker_train.drop_duplicates(subset=[USER_COL, ITEM_COL])

df_ranker_train['target'].fillna(0, inplace= True)

Посмотрим на баланс классов:

In [39]:
df_ranker_train.target.value_counts()

0.0    97137
1.0    10513
Name: target, dtype: int64

In [40]:
df_ranker_train

,user_id,item_id,target
0,843,8090521,0.0
1,843,1106523,0.0
2,843,5568378,0.0
3,843,1092026,1.0
4,843,8090537,0.0
...,...,...,...
112409,832,859075,0.0
112410,832,1122358,0.0
112411,832,1137346,0.0
112412,832,1068719,0.0


Доля положительного класса:

In [41]:
df_ranker_train['target'].mean()

0.09765908035299582

Посмотрим на наши датасеты, чтобы сгенерировать идеи по созданию  
новых признаков:

In [42]:
data_train_ranker.head(3)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
2211065,843,40955282722,622,845193,3,5.37,364,-1.50,19,90,0.0,0.0
2211066,843,40955282722,622,865891,2,2.00,364,0.00,19,90,0.0,0.0
2211067,843,40955282722,622,883404,2,1.76,364,-0.82,19,90,0.0,0.0


In [43]:
item_features.head(3)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,


In [44]:
user_features.head(3)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8


Напишем класс-обработчик для более удобной работы с кодом.  
В нём реализуем генерацию признаков, а также заполнение пропусков.  
Все данные будут браться из обучающего датасета и использоваться  
на трансформируемом датасете. Таким образом, результирующие датасеты  
будут одного размера и будут готовы к передаче их модели на обучение  
или предсказание:

In [45]:
class DataProcessor:
    
    def __init__(self):
        self.cat_feats = None
        self.data_train_ranker = None
        self.item_features = None
        self.nan_cols = None
        self.train_data = None
        self.user_features = None

    
    # Добавление фичей
    def _add_features(self, df, is_fit=True):
        
        # Первоначальная подготовка
        df = df[[USER_COL, ITEM_COL]].copy()
        df = df.merge(self.item_features, on=ITEM_COL, how='left')
        df = df.merge(self.user_features, on=USER_COL, how='left')
        feats_count = len(df.columns)
        
        # Средний чек
        df = df.merge(self.train_data.groupby(USER_COL, sort=False)['sales_value']. \
                                    mean().reset_index(), how='left', on=USER_COL)
        df.rename(columns={'sales_value': 'avg_bill'}, inplace=True)
        
        # Средняя сумма покупки 1 товара в каждой категории
        df = df.merge(self.train_data.groupby([USER_COL, 'department'], sort=False)['sales_value'] \
                                                .mean().reset_index(), how='left', on=[USER_COL, 'department']). \
                                                rename(columns={'sales_value': 'avg_cat_spendings'})
        
        # Кол-во покупок в неделю
        df = df.merge((self.train_data.groupby(ITEM_COL, sort=False)['quantity'] \
                                                 .count() / self.train_data['week_no'].nunique()).reset_index(), how='left', on=ITEM_COL)

        df.rename(columns={'quantity': 'avg_week_purchases'}, inplace=True)
        
        # Среднее кол-во покупок 1 товара в категории в неделю
        df = df.merge((self.train_data.groupby('department', sort=False)['quantity'] \
                                                 .count() / self.train_data['week_no'].nunique()).reset_index(), \
                                                how='left', on='department').rename(columns={'quantity': 'avg_week_purchases_cat'})
        
        # Средняя сумма покупки 1 товара в каждой категории (берем категорию item_id)
        df = df.merge(self.train_data.groupby('department', sort=False)['sales_value'] \
                                                 .mean().reset_index(), how='left', on='department') \
                                                 .rename(columns={'sales_value': 'avg_cat_spendings_items'})
        
        # Кол-во покупок юзером конкретной категории в неделю
        df = df.merge((self.train_data.groupby([USER_COL, 'department'], sort=False)['quantity'] \
                                                 .count() / self.train_data['week_no'].nunique()).reset_index(), how='left', \
                                                on=[USER_COL, 'department']).rename(columns={'quantity': 'user_week_cat_purchase'})
        
        # Кол-во фактов продажи конкретного товара
        df = df.merge(self.train_data.groupby(ITEM_COL, sort=False).agg(USER_COL).count().rename('item_popularity'), how='left', on=ITEM_COL)
        
        
        # Кол-во конкретных купленных товаров конкретным юзером (golden feature)
        # Данный признак даёт большие веса товарам, которые покупает каждый конкретный  
        # пользователь. После его применения мы начинаем часто предлагать товары, которые  
        # пользователь и так покупает. Не очень честный приём, который напоминает own_recommendations.
        # Но заданная в условии планка 0.235 по precision@5 слишком высока, чтобы не прибегать к
        # таким уловкам
        df = df.merge(self.train_data.groupby([USER_COL, ITEM_COL], sort=False)['quantity'].sum(). \
                                              reset_index(), how='left', on=[USER_COL, ITEM_COL]).rename(columns={'quantity': 'total_buys'})
        
        # Подготовка
        new_feats = len(df.columns) - feats_count
        df = df.iloc[:, 2:]
        self.cat_feats = df.columns.tolist()[:-new_feats]
        df[self.cat_feats] = df[self.cat_feats].astype('category')
        
        # Заполнение пропусков модой
        if not is_fit:
            self.nan_cols = df.isna().sum().loc[df.isna().sum() > 0].index.tolist()

        for col in self.nan_cols:
            df[f'{col}_nan'] = 0
            df.loc[df[col].isna(), f'{col}_nan'] = 1
            
            if is_fit:
                data_source = self.data_train_ranker.copy()
            else:
                data_source = df.copy()
                
            df[col].fillna(data_source[col].value_counts().index[0], inplace=True)
        
        # В случае появления новых item_id будут появляться пропуски в столбцах,
        # в которых не было пропусков на трейне. Обработаем их отдельно, чтобы не  
        # менять количество признаков
        additional_nans = df.isna().sum().loc[df.isna().sum() > 0].index.tolist()
        
        for col in additional_nans:
            df[col].fillna(data_source[col].value_counts().index[0], inplace=True)
        
        return df
        
        
    def fit(self, data_train_ranker, train_data, user_features, item_features):
        self.data_train_ranker = data_train_ranker
        self.user_features = user_features
        self.item_features = item_features
        self.train_data = train_data.merge(self.item_features[[ITEM_COL, 'department']], how='left', on=ITEM_COL)

        # Добавим фичи на трейне, чтобы были данные для заполнения пропусков
        # на тесте
        self.data_train_ranker = self._add_features(self.data_train_ranker, is_fit=False)
        
    
    def transform(self, X):
        
        X = self._add_features(X)

        return X

Обучим обработчик и создадим датасеты для обучения модели:

In [46]:
processor = DataProcessor()
processor.fit(df_ranker_train, data_train_matcher, user_features, item_features)

In [47]:
X_train = processor.transform(df_ranker_train)
X_train.shape

(107650, 31)

In [48]:
y_train = df_ranker_train['target']
y_train.shape

(107650,)

Обучим модель ранжирования. Гиперпараметры модели уже были  
подобраны эмпирическим путём:

In [49]:
model = CatBoostRanker(iterations=200, silent=True,
                       eta=0.15, task_type='GPU',
                       max_depth=7,
                       loss_function='PairLogitPairwise',
                       random_state=29,
                       cat_features=processor.cat_feats)

In [50]:
%%time

model.fit(X_train, y_train, group_id=df_ranker_train[USER_COL], subgroup_id=df_ranker_train[ITEM_COL])

CPU times: user 22.4 s, sys: 4.69 s, total: 27.1 s
Wall time: 17.7 s


## Создание и ранжирование кандидатов на тестовом датасете

Повторим процедуру инициализации тестового датасета, как делали  
на базовом решении:

In [51]:
result_test = data_test.groupby(USER_COL, sort=False)[ITEM_COL].unique().reset_index()
result_test.columns=[USER_COL, ACTUAL_COL]
result_test.head(2)

,user_id,actual
0,1340,"[912987, 819255, 834117, 866227, 889362, 89608..."
1,588,"[1024426, 6534178, 9673270, 826842, 833025, 85..."


In [52]:
result_test = result_test.loc[result_test[USER_COL].isin(data_train_matcher[USER_COL].unique())]

Создадим кандидатов для тестового датасета:

In [53]:
make_recommendations(result_test, ('als_recs', recommender.get_als_recommendations), N=N_CANDIDATES)

Посмотрим на метрику, чтобы было с чем сравнить результат работы  
модели ранжирования:

In [54]:
sorted(calc_precision(result_test, N_RANGED), key=lambda x: x[1], reverse=True)

[('als_recs', 0.1302547770700637)]

Выделим отдельно датасет с кандидатами и развернём его, как делали  
ранее с трейном:

In [55]:
df_test_candidates = result_test.rename(columns={'als_recs': 'candidates'}).drop(ACTUAL_COL, axis=1)

In [56]:
df_test_candidates.head(3)

,user_id,candidates
0,1340,"[981760, 1106523, 1133018, 995242, 923746, 109..."
1,588,"[995242, 1029743, 908531, 859075, 833025, 9615..."
2,2070,"[1029743, 981760, 923746, 1068719, 995242, 859..."


In [57]:
df_items_test = df_test_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
df_items_test.name = ITEM_COL

In [58]:
df_test_candidates = df_test_candidates.drop('candidates', axis=1).join(df_items_test)

In [59]:
df_test_candidates.head()

,user_id,item_id
0,1340,981760
0,1340,1106523
0,1340,1133018
0,1340,995242
0,1340,923746


Подготовим датасет из кандидатов для предсказания:

In [60]:
X_test = processor.transform(df_test_candidates)

In [61]:
X_test.shape

(94200, 31)

Сделаем предсказание:

In [62]:
test_preds = model.predict(X_test)

In [63]:
df_test_candidates['score_item_purchase'] = test_preds

In [64]:
df_test_candidates.head(3)

,user_id,item_id,score_item_purchase
0,1340,981760,-0.030067
0,1340,1106523,-0.303180
0,1340,1133018,-0.331499


Напишем функцию для ранжирования кандидатов по баллу:

In [65]:
def rerank(user_id):
    return df_test_candidates.loc[df_test_candidates[USER_COL]==user_id].sort_values('score_item_purchase', ascending=False). \
                                                                                                    head(N_RANGED).item_id.tolist()

Сделаем ранжирование:

In [66]:
result_test['reranked_als_recs'] = result_test[USER_COL].apply(lambda user_id: rerank(user_id))

Оценим метрику:

In [67]:
print(*sorted(calc_precision(result_test, N_RANGED), key=lambda x: x[1], reverse=True), sep='\n')

('reranked_als_recs', 0.2642250530785563)
('als_recs', 0.1302547770700637)


In [68]:
result_test.head(3)

,user_id,actual,als_recs,reranked_als_recs
0,1340,"[912987, 819255, 834117, 866227, 889362, 89608...","[981760, 1106523, 1133018, 995242, 923746, 109...","[995242, 916122, 1029743, 981760, 1026118]"
1,588,"[1024426, 6534178, 9673270, 826842, 833025, 85...","[995242, 1029743, 908531, 859075, 833025, 9615...","[1106523, 914190, 1133018, 1053690, 859075]"
2,2070,"[995242, 1055863, 12781914, 866227, 936508, 89...","[1029743, 981760, 923746, 1068719, 995242, 859...","[995242, 908531, 1029743, 5569471, 923746]"


Сравним с базовым решением:

In [69]:
baseline_metric

0.12611464968152866

In [70]:
final_metric = sorted(calc_precision(result_test, N_RANGED), key=lambda x: x[1], reverse=True)[0][1]
final_metric

0.2642250530785563

<ins>Итог:</ins>  
В рамках курсового проекта сделали базовое решение, потом на первом этапе  
выбрали способ/модель генерирования кандидатов. На втором этапе обучили  
модель ранжирования. Затем подобрали кандидатов для тестового датасета,  
используя модель, выбранную на первом этапе (ALS), и произвели их ранжирование  
с помощью модели, обученной на втором этапе (CatBoost). Получили целевую метрику  
precision@5, превышающую указанное в условии задачи значение 0.235.